# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

## Подготовка

Загружаем нужные библиотеки

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.8 MB/s eta 0:00:00


In [2]:
!pip install detoxify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 101.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.0
    Uninstalling transformers-4.26.0:
      Successfully uninstalled transformers-4.26.0


In [3]:
import pandas as pd
import numpy as np

import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, PredefinedSplit
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings('ignore')

import torch
import transformers as ppb
from detoxify import Detoxify

from lightgbm import LGBMClassifier
import lightgbm as lgb
from scipy import sparse

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Прочтем наш датасет и выведем на экран первые 5 строк.

In [21]:
try:
    df = pd.read_csv('/content/toxic_comments.csv', error_bad_lines=False, engine='python', index_col=0)
except FileNotFoundError:
    df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
df.head()

Skipping line 123440: unexpected end of data


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отлично, данные загружены:) </div>

Посмотрим общую информацию о датасете

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123438 entries, 0 to 123565
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    123438 non-null  object
 1   toxic   123438 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.8+ MB


Датасет содержит 159451 строку и 2 столбца. 
В столбце toxic есть пропуски. А также этот столбец имеет формат object, нам нужно перевести его в целочисленный формат.

Посмотрим есть ли в датасете дубликаты. Если есть, то удалим их.

In [23]:
df.duplicated().sum()

0

Посмотрим какие значения есть в столбце toxic

In [25]:
df['toxic'].unique()

array([0, 1])

Посмотрим какое соотношение негативных и позитивных комментариев в столбце toxic

In [26]:
print(df['toxic'].value_counts())
print('Соотношение позитивных комментариев к негативным:', df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1])

0    110880
1     12558
Name: toxic, dtype: int64
Соотношение позитивных комментариев к негативным: 8.82943143812709


Классы несбалансированы. Позитивных комментарием почти в 10 раз больше, чем негативных. В процессе обучения модели протестируем способы балансировки классов.

Сделаем копию подготовленного датасета для работы с алгоритмом BERT.

In [27]:
df_bert = df.copy()
df_bert.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Выводы:
- Исходных датасет содержит 2 столбца (текстовый комментарий и оценку комментария);
- В процессе подготовки датасета удалили дубликаты и пропуски из датасета. А также текстовые ячейки в столбце оценки. 
- Оценка комментариев содержит дисбаланс классов. В процессе обучения модели поработаем над этим. 

## Обучение

### Подготовка признаков

Проведем лемматизацию столбца text  и очистку  от регулярных выражений. 

In [9]:
# Загрузим словари
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

# Напишем функцию лемматизации и очистки.
wnl = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    word_list = nltk.word_tokenize(text)
    lemm_word = ' '.join([wnl.lemmatize(w) for w in word_list])
    #"".join(wnl.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_word) 
    return " ".join(cleared_text.split())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Создадим столбец lemm_text с обработанным текстом
df['lemm_text'] = df['text'].apply(lemmatize_text)

# Отслеживаем выполнение фунции. 
tqdm.pandas()
df['text'] = df['text'].progress_apply(lemmatize_text) 

100%|██████████| 159292/159292 [02:42<00:00, 977.48it/s]


In [ ]:
# удалим столбец text
df = df.drop(['text'], axis=1)

In [ ]:
df.head()

,toxic,lemm_text
0,0,explanation why the edits made under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not trying to edit war it s...
3,0,more i ca n t make any real suggestion on impr...
4,0,you sir are my hero any chance you remember wh...


Создадим переменные с признаками и целевым признаком, разделим их на тренировочную и тестовую выборки.

In [ ]:
target = df['toxic']
features = df['lemm_text']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

In [ ]:
features_train.shape

(119469,)

In [ ]:
target_train.shape

(119469,)

### Логистическая регрессия

Для работы с тестами при работе с классификацией лучшим образом подходит модель логистической регрессии. 

Т.к. у нас дисбаланс класов, то используем в параметрах balanced.  Определим наилучшие параметры.

In [ ]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=1234)),
])

parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': range(1, 10)
}

grid_search_tune = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(features_train, target_train)
grid_search_tune.best_params_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


{'logreg__C': 9, 'tfidf__ngram_range': (1, 1)}

Обучим модель с наилучшими параметрами и определим качество модели.

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train_logreg = count_tf_idf.fit_transform(features_train)
features_test_logreg = count_tf_idf.transform(features_test)

model = LogisticRegression(random_state=1234, C=9)
model.fit(features_train_logreg, target_train)
predicted = model.predict(features_test_logreg)
f1_scoring = f1_score(target_test, predicted)
f1_scoring

0.7847154921777655

#### Работа с дисбалансом.

Параметр class_weight = balansed мы уже протестировали. Попробуем ресемплирование с уменьшением класса.

F1 с балансом классов ниже, чем у модели без баланса. Попробуем ресемплировать классы.

**Ресемплирование с уменьшением класса 0.**

Выделим из тренировочного датафрейма валидную и тренеровочную часть. Затем проведем балансировку тренировочного датасета.

In [ ]:
# выделим тренировочный датафрейм из нашего датасета.
df_train = df.loc[target_train.index]

# разделим тренировочный датасет на обучающую и валидационную выборку. 

df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=12345)
df_train.shape

(89601, 2)

In [ ]:
target_train_class_zero = df_train[df_train['toxic'] == 0]['toxic']
target_train_class_one = df_train[df_train['toxic'] == 1]['toxic']

In [ ]:
target_train_class_zero_downsample = target_train_class_zero.sample(target_train_class_one.shape[0],
                                                                    random_state=1234)
target_train_downsample = pd.concat([target_train_class_zero_downsample, target_train_class_one])

features_train_downsample = df_train['lemm_text'].loc[target_train_downsample.index]
features_train_downsample, target_train_downsample = shuffle(features_train_downsample,
                                                             target_train_downsample,
                                                             random_state=12345)
# Вычислим TF-IDF для текстов.
#features_train_downsample = count_tf_idf.transform(features_train_downsample['lemm_text']
#                                                   .values.astype('U'))

In [ ]:
# создаем переменную с признаками и целевым признаком для валидационной выборки.
valid_features = df_valid['lemm_text']
valid_target = df_valid['toxic']

In [ ]:
valid_features.shape

(29868,)

In [ ]:
features_train_downsample.shape

(18190,)

Обучим модель, используя новые данные.

In [ ]:
X  = pd.concat([features_train_downsample, valid_features]) 
test_fold = [-1 for _ in range(valid_features.shape[0])] + [0 for _ in range(features_train_downsample.shape[0])]
y = np.concatenate([target_train_downsample, valid_target])
ps = PredefinedSplit(test_fold)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=1234)),
])

parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': range(1, 10)
}

grid_search_tune = GridSearchCV(pipeline, parameters, cv=ps, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(X, y)
grid_search_tune.best_params_

Fitting 1 folds for each of 27 candidates, totalling 27 fits


{'logreg__C': 2, 'tfidf__ngram_range': (1, 1)}

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train_downsample = count_tf_idf.fit_transform(features_train_downsample)
features_test_resemple = count_tf_idf.transform(features_test)

model = LogisticRegression(random_state=1234, C=2)
model.fit(features_train_downsample,target_train_downsample)
predicted = model.predict(features_test_resemple)
f1_scoring_resemple = f1_score(target_test, predicted)
f1_scoring_resemple
print('F1 Логистической регрессии с уменьшением классов', f1_scoring_resemple)

F1 Логистической регрессии с уменьшением классов 0.6706163401815577


Значение f1 при балансировании классов ниже, чем при встроенном параметре "balansed" логистической регрессии.

## Стохастический градиентный спуск SGDCClassifier.

Протестируем модель.

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('SGDC', SGDClassifier(random_state=1234)),
])

parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'SGDC__loss': ['hinge', 'log', 'modified_huber'],
    'SGDC__eta0': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5]
}

grid_search_tune = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(features_train, target_train)
grid_search_tune.best_params_

Fitting 3 folds for each of 54 candidates, totalling 162 fits


{'SGDC__eta0': 0.01,
 'SGDC__loss': 'modified_huber',
 'tfidf__ngram_range': (1, 1)}

In [ ]:
SGDClassifier(random_state=1234).get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [ ]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'tfidf', 'SGDC', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary', 'SGDC__alpha', 'SGDC__average', 'SGDC__class_weight', 'SGDC__early_stopping', 'SGDC__epsilon', 'SGDC__eta0', 'SGDC__fit_intercept', 'SGDC__l1_ratio', 'SGDC__learning_rate', 'SGDC__loss', 'SGDC__max_iter', 'SGDC__n_iter_no_change', 'SGDC__n_jobs', 'SGDC__penalty', 'SGDC__power_t', 'SGDC__random_state', 'SGDC__shuffle', 'SGDC__tol', 'SGDC__validation_fraction', 'SGDC__verbose', 'SGDC__warm_start'])

Проверяем значение F1.

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train_sgdc = count_tf_idf.fit_transform(features_train)
features_test_sgdc = count_tf_idf.transform(features_test)

model = SGDClassifier(random_state=1234, eta0=0.01, loss='modified_huber')
model.fit(features_train_sgdc,target_train)
predicted = model.predict(features_test_sgdc)
f1_scoring_sgdc = f1_score(target_test, predicted)
f1_scoring_sgdc
print('F1 SGDCClassifier', f1_scoring_sgdc)

F1 SGDCClassifier 0.7366832654293044


**Итоги**

Сведем результаты обучений в одну таблицу.

In [ ]:
columns = ['Модель', 'F1']
model_lr = ['Логистическая регрессия', f1_scoring]
model_lr_balanced = ['Логистическая регрессия с балансом классов', f1_scoring_resemple]
model_sgdc = ['Стохастический градиентный спуск SGDCClassifier', f1_scoring_sgdc]

table = pd.DataFrame([model_lr, model_lr_balanced, model_sgdc], columns = columns)
display(table)

,Модель,F1
0,Логистическая регрессия,0.784715
1,Логистическая регрессия с балансом классов,0.670616
2,Стохастический градиентный спуск SGDCClassifier,0.736683


### Обучение через BERT

Для работы сменим среду на GPU.

In [28]:
dev = 'cuda:0'
device = torch.device(dev)

Возьмем выборку из 60000 строк.

In [29]:
batch_1 = df_bert.sample(60000).reset_index(drop=True)
batch_1.head()

,text,toxic
0,March 2008 (UTC)\n___\n[Image:Information.svg|...,0
1,(Again giving Sadasivan as Ref),0
2,I documented the change I made on Talk. The 'o...,0
3,"BUSH US EVIL, HE'S SELFISH AND VERY SHORT TEMP...",1
4,in your sandbox dog gave out a shit,1


Посмотрим соотношение позитивных и негативных оценок.

In [30]:
print(batch_1['toxic'].value_counts())
batch_1['toxic'].value_counts()[0] / batch_1['toxic'].value_counts()[1]

0    53910
1     6090
Name: toxic, dtype: int64


8.852216748768473

Импортируем токенизатор

In [31]:

model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Проводим токенизацию текста нашего датасета

In [ ]:
tokenized = batch_1['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)))

Приводим векторы к одному размеру.


In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

Создадим маску для значимых токенов.

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

Обработка в DistilBERT'е

Теперь создадим входной вектор из матрицы токенов и передадим его в BERT.

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)
#with torch.no_grad():
#  last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
%%time
from tqdm import notebook
batch_size = 200 # для примера возьмем такой батч, где будет всего две строки датасета (возьми сколько влезет на GPU), чем больше батч, тем быстрее пройдет обработка
embeddings = [] 
for i in notebook.tqdm(range(input_ids.shape[0] // batch_size)):
        batch = torch.LongTensor(input_ids[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings

In [ ]:
features = np.concatenate(embeddings)
target = batch_1['toxic']

train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.25, random_state=12345)

Обучаем Логистическую регрессию.

In [ ]:
parameters = {'C': range(1, 10),
              'class_weight': ['None', 'balansed']
              }
grid_search = GridSearchCV(LogisticRegression(), parameters, scoring='f1', cv=3)
grid_search.fit(train_features, train_target)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 6, 'class_weight': 'None'}
best scrores:  0.708947599405685


In [ ]:
model = LogisticRegression(random_state=1234, C=6)
model.fit(train_features, train_target)
predict = model.predict(test_features)
score = f1_score(test_target, predict)
score

0.7098143236074269

## Выводы:

Заказчик магазин "Викишоп" запускает новый сервис, который позвозяет пользователями редактировать и дополнять описание товаров. Необходимо было подобрать эффективную модель машинного обучения, которая поможет классифицировать комментарии пользователей на позитивные и негативные.

В ходе работы с датасетом было сделано:

- Подготовленны данные исходного датасета.
- Проведена работа над балансом классов.
- Перед обучением проведена обработка теста (лемматизация, очистка, расчет TFIDF).
- Выбрана лучшая модель.
- Показаны параметры качества моделей.

Наилучшие результаты показали работа с BERT. 
На тестовой выбоке метрика F1 - 0.7098